In [1]:
import pandas as pd
import os
import sys
import glob
import datetime as dt
import pyodbc
import numpy as np
import re
import sys


In [2]:
def SplitsFirstTwo(string):
    return string[0:2]
#C:\Users\mohamed.sultan\Migration\DataFiles

In [3]:
def DateExtract(string):
    return string[2:10]

In [4]:
#C:\Users\mohamed.sultan\Migration\DataFiles

In [5]:
print("Please enter yor access files path") 
Path=r"D:\work\Work\Migration\25_10"
#Path=input()

Please enter yor access files path


In [6]:
date=dt.datetime.now()
ConnectionDBParamter=r'Driver={Microsoft Access Driver (*.mdb, *.accdb)}'
column_names={'FirmID':'FACILITY_ID','FirmName':'NAME','FirmAddress':'Detailed_ADDRESS','LicenseCode':'LICENSE_Number',
              'GovID':'GOVERNORATE','City':'CITY','CompanyName':'Name','LegalEntityID':'Type','CommName':'COMM_NAME','PowerHorse':'SUM_MACHINE_DFORCE_HP','Area':'TOTAL_AREA','LicenseServiceID':'License_ID',
             'IssuedDate':'ISSUE_DATE','ExpiaryDate':'LIC_ISSUE_END_DATE','ServiceID':'ServiceID',
             'IndusterialActivityCode':'ACTIVITY_ID'}
sql="SELECT Licenses.FirmID, Licenses.FirmName, Licenses.FirmAddress, Licenses.LicenseCode, Licenses.GovID, Licenses.City, Licenses.CompanyName, Licenses.LegalEntityID, Licenses.CommName, Licenses.PowerHorse, Licenses.Area, LicenseService.LicenseServiceID, LicenseService.IssuedDate, LicenseService.ExpiaryDate, LicenseService.ServiceID, IndusterialActivities.IndusterialActivityCode FROM (Licenses INNER JOIN LicenseService ON Licenses.FirmID = LicenseService.FirmID) INNER JOIN IndusterialActivities ON LicenseService.LicenseServiceID = IndusterialActivities.LicenseServiceID;"
column_names=['FACILITY_ID','NAME','Detailed_ADDRESS','LICENSE_Number','GOVERNORATE','CITY','Name','Type','COMM_NAME','SUM_MACHINE_DFORCE_HP','TOTAL_AREA','License_ID','ISSUE_DATE','LIC_ISSUE_END_DATE','ServiceID','ACTIVITY_ID']
df=pd.DataFrame()


In [7]:
for filepath in glob.iglob(r'{0}\*.accdb'.format(Path)):
    
    conn = pyodbc.connect(r'{0};DBQ={1};'.format(ConnectionDBParamter,filepath))

#    SQL_Query = pd.read_sql_query('''SELECT * from MigrationView;''', conn,)
    SQL_Query = pd.read_sql_query(sql, conn,)
    if df.empty:
        df=pd.DataFrame(SQL_Query)
       
    else:
        df=pd.concat([df,pd.DataFrame(SQL_Query)],ignore_index=True,axis=0)
        #df=pd.concat([df,pd.DataFrame(SQL_Query, columns=column_names)],ignore_index=True,axis=0)
   
  

In [8]:

df["BranchID"]=df["LicenseCode"].apply(SplitsFirstTwo)
df["Date"]=df["LicenseCode"].apply(DateExtract)


In [9]:
df["Date"]=pd.to_datetime(df["Date"],format=r"%Y%m%d")
df =df[df["Date"]<="2020/10/25"]


In [10]:
df['GovID'].fillna(0,inplace=True)
df['GovID'] = df['GovID'].astype(int).astype(str)
govdict={"18":"60","17":"18","1":"17","3":"1","2":"3","60":"2"}
df['GovID'].replace(govdict,inplace=True)


In [15]:
import pandas as pd
import os
import sys
import glob
import datetime as dt
import pyodbc
import numpy as np
import re
import sys
df=pd.read_excel(r"D:\dropbox\Dropbox\ExportLicence.xls")


In [16]:
df1=pd.read_excel(r"D:\dropbox\Dropbox\MigrationQuerysegel25-10-2020.xlsx")
df1.columns

Index(['seg_cd', 'Product_Nm', 'Product_Cd', 'prod_unit', 'prod_quantity',
       'LEG_NM', 'certkind', 'firms', 'factory_location', 'seg_issue_date',
       'seg_end_date', 'tel', 'fax', 'Cond', 'adminfirm', 'CommercialFeature'],
      dtype='object')

In [18]:
df.columns
df['LicenseCode'] = df['LicenseCode'].astype(int).astype(str)

In [28]:
df1.dtypes

seg_cd                       object
Product_Nm                   object
Product_Cd                   object
prod_unit                    object
prod_quantity               float64
LEG_NM                      float64
certkind                     object
firms                        object
factory_location             object
seg_issue_date       datetime64[ns]
seg_end_date         datetime64[ns]
tel                          object
fax                          object
Cond                         object
adminfirm                    object
CommercialFeature            object
dtype: object

In [29]:
def Trimandclearasscents(Dataframe,FieldsList,ConcatList):
    DataframeSub=Dataframe[FieldsList]
    for field in FieldsList:
        #DataframeSub[field].replace({None:' '},inplace=True,regex=True)
        DataframeSub[field].replace({"NaN":' '},inplace=True,regex=True)
        DataframeSub[field].replace({'[إأآا]':'ا'},inplace=True,regex=True)
        DataframeSub[field].replace({'ؤ':'ء'},inplace=True,regex=True)
        DataframeSub[field].replace({'ئ':'ء'},inplace=True,regex=True)
        DataframeSub[field].replace({'گ':'ك'},inplace=True,regex=True)
    DataframeSub["NameCompare"]=DataframeSub[ConcatList[0]]+ " "+DataframeSub[ConcatList[1]]
    DataframeSub["NameCompare"]=DataframeSub["NameCompare"].astype(str)
    DataframeSub.drop(ConcatList, axis='columns', inplace=True)
    DataframeSub["NameCompare_len"]=DataframeSub["NameCompare"].apply(len)
    if "FirmAddress" in FieldsList:
        DataframeSub["Address_Licence_len"]=DataframeSub["FirmAddress"].apply(len)
        DataframeSub.rename({"NameCompare":"NameCompare_Licence","FirmAddress":"Address_Licence"},axis='columns',inplace=True)
    if "factory_location" in FieldsList:
        DataframeSub["factory_location"]=DataframeSub["factory_location"].astype(str)
        DataframeSub["Address_segl_len"]=DataframeSub["factory_location"].apply(len)
        DataframeSub.rename({"NameCompare":"NameCompare_segl","factory_location":"Address_segl"},axis='columns',inplace=True)   
    return DataframeSub
MainLicenseFieldsdf=Trimandclearasscents(df,["FirmName","FirmAddress","LicenseCode","CompanyName"],["CompanyName","FirmName"]) 
MainSegelFieldsdf=Trimandclearasscents(df1,["firms","factory_location","seg_cd",'CommercialFeature'],['CommercialFeature',"firms"])

<ipython-input-29-93559ca1a0f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataframeSub["NameCompare"]=DataframeSub[ConcatList[0]]+ " "+DataframeSub[ConcatList[1]]
<ipython-input-29-93559ca1a0f2>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataframeSub["NameCompare"]=DataframeSub["NameCompare"].astype(str)
<ipython-input-29-93559ca1a0f2>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [41]:
del MainSegelFieldsdf

In [44]:
#del MainLicenseFieldsdf
del df
#del df1

In [32]:
#or sort and remove duplictae and keep the highest 
def MaxConcat(Dataframe,GroupbyField,ConcatListdict):
    grp=Dataframe.groupby(GroupbyField)
    grpdict={}
    result=pd.DataFrame(Dataframe[GroupbyField].drop_duplicates())
    for key,valuelist in ConcatListdict.items():
        grpdict[key]= pd.DataFrame(grp[valuelist].max()).reset_index() 
    for key,Table in grpdict.items():  
        result = pd.merge(result, Table, on=GroupbyField)
    result["Key"]=1
    return result
Licensecleaned=MaxConcat(MainLicenseFieldsdf,"LicenseCode",{"Name":["NameCompare_len","NameCompare_Licence"],"Address":["Address_Licence_len","Address_Licence"]})
Segelcleaned=MaxConcat(MainSegelFieldsdf,"seg_cd",{"Name":["NameCompare_len","NameCompare_segl"],"Address":["Address_segl_len","Address_segl"]})


In [1]:
Licensecleaned=Licensecleaned[["LicenseCode","Address_Licence","NameCompare_Licence","Key"]]
Segelcleaned=Segelcleaned[["seg_cd","Address_segl","NameCompare_segl","Key"]]


NameError: name 'Licensecleaned' is not defined

In [ ]:
import pandas as pd
import os
import sys
import glob
import datetime as dt
import pyodbc
import numpy as np
import re
import sys



from pandas import DataFrame, merge
df1 = DataFrame({'key':[1,1], 'col1':[1,2],'col2':[3,4]})
df2 = DataFrame({'key':[1,1], 'col3':[5,6]})

merge(df1, df2,on='key')[['col1', 'col2', 'col3']]

In [34]:
Licensecleaned

,LicenseCode,Address_Licence,NameCompare_Licence,Key
0,1114326625,محافظة اسوان - مركز ادفو - الفوزة- بملك / نفي...,احمد محمد احمد حامد مخبز افرنجى,1
1,1114326626,محافظة اسوان/ المنطقة الصناعية طريق اسوان العلاقى,nan,1
2,2003726625,محافظة اسوان/ مركز كوم امبو - الرغامه شرق,nan,1
3,2003726626,محافظة اسوان / مركز نصر النوبة - ارمنا,nan,1
4,2003726627,محافظة اسوان / مدينة ابوسمبل السياحية -قسم ابو...,nan,1
...,...,...,...,...
33170,-265441917,قطعة رقم ( 1-52 ) بمنطقة الصناعات المتوسطة - م...,nan,1
33171,-265441916,قطعة رقم 91 منطقة الصناعات الخفيفة - مدينة بنى...,nan,1
33172,-265341919,قطعة (42) بال ( 75 ) فدان المنطقة الصناعية - ا...,nan,1
33173,-265241919,قطعة رقم 121 بامتداد المنطقة الصناعية الثالثة ...,nan,1


In [35]:
Segelcleaned

,seg_cd,Address_segl,NameCompare_segl,Key
0,2016/41930,مصنع مءجر بملك / سعيد احمد معوض عبد الباقى - ا...,nan,1
1,2005/31258,كفر سعد/ اول طريق بنها المنصورة/ بنها/ محافظة ...,nan,1
2,2005/31063,26 شارع سعد زغلول من شارع 135 طريق بهتيم / شبر...,nan,1
3,2009/34162,مصنع مءجر مقام على القطعة رقم1/ 18/بلوك20016/ا...,nan,1
4,2007/32080,القطعة رقم 8/1/ الوحدة 3/ مجمع الشركة السويسري...,nan,1
...,...,...,...,...
14075,2018/45093,مخبز مءجر بملك العربى عنتر حسين / ب 10 مسجد ا...,nan,1
14076,2020/47183,مصنع مءجر / قطعة رقم 191 / منطقة الصناعات المت...,nan,1
14077,2019/45837,قطعة رقم 212 / بقطاع الصناعات الغذاءية / بالمن...,اورجنك لايف للصناعات الغذاءية اورجنك لايف للصن...,1
14078,2007/32378,عنبر مءجر بالدور الثاني علوي من المبني المقام ...,المصرية للالياف المعدنية المصرية للالياف المعدنية,1


In [1]:
import pandas as pd
import os
import sys
import glob
import datetime as dt
import pyodbc
import numpy as np
import re
import sys
Segelcleaned=pd.read_excel(r"D:\dropbox\Dropbox\DataScience\Segelcleaned.xls")
Licensecleaned=pd.read_excel(r"D:\dropbox\Dropbox\DataScience\Licensecleaned.xls")

In [27]:
Segelcleaned.shape

(33175, 5)

In [2]:
#create Crossproduct forcomparizon
FuzzySet =pd.merge(Licensecleaned, Segelcleaned,on='Key')[["seg_cd","Address_segl","NameCompare_segl","LicenseCode","Address_Licence","NameCompare_Licence"]]

MemoryError: Unable to allocate 10.4 GiB for an array with shape (3, 467137175) and data type int64

In [2]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

    

In [5]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=1):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
   

    df_1['matches'] = df_1[key1].apply(lambda x: process.extractOne(x, s))  
     

    #df_1['matches'] = df_1['matches'].apply(lambda x: ', '.join(x))
    

    return df_1

In [ ]:

FuzzySet=fuzzy_merge(Licensecleaned,Segelcleaned,"NameCompare_Licence","NameCompare_segl")
FuzzySet

In [ ]:
def maxi(id):
    max()
    
    

In [ ]:
df.to_excel("ExportLicence.xls")

In [10]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
import os
import sys
import glob
import datetime as dt
import pyodbc
import numpy as np
import re
import sys
import pandas as pd
from pathlib import Path
import fuzzymatcher 

In [5]:
Segelcleaned=pd.read_excel(r"D:\dropbox\Dropbox\DataScience\Segelcleaned.xls")
Licensecleaned=pd.read_excel(r"D:\dropbox\Dropbox\DataScience\Licensecleaned.xls")

In [29]:
Segelcleaned.shape

(14081, 5)

In [30]:
Licensecleaned.shape

(33175, 5)

In [6]:
left_on = ["NameCompare_Licence", "Address_Licence"]

right_on = ["NameCompare_segl", "Address_segl"]

In [11]:
matched_results = fuzzymatcher.fuzzy_left_join(Licensecleaned,
                                            Segelcleaned,
                                            left_on,
                                            right_on,
                                            left_id_col='LicenseCode',
                                            right_id_col='seg_cd')

In [20]:
matched_results.to_excel("matched_results.xls")

In [18]:
matched_results["LicenseCode"]=matched_results["LicenseCode"].astype(str)

In [19]:
matched_results.dtypes

best_match_score       float64
__id_left                int64
__id_right              object
Unnamed: 0_left          int64
LicenseCode             object
Address_Licence         object
NameCompare_Licence     object
Key_left                 int64
Unnamed: 0_right       float64
seg_cd                  object
Address_segl            object
NameCompare_segl        object
Key_right              float64
dtype: object

In [21]:
matched_results.shape

(33175, 13)

In [27]:
matched_results["LicenseCode"].duplicated()

<bound method Series.sum of 0          False
262        False
490        False
712        False
947        False
           ...  
7525364    False
7525588    False
7525816    False
7526079    False
7526296    False
Name: LicenseCode, Length: 33175, dtype: bool>

In [53]:
matched_results[(matched_results["best_match_score"]>.1) & (matched_results["best_match_score"]<.3)]

,best_match_score,__id_left,__id_right,Unnamed: 0_left,LicenseCode,Address_Licence,NameCompare_Licence,Key_left,Unnamed: 0_right,seg_cd,Address_segl,NameCompare_segl,Key_right
64934,0.318419,532017112300002,2017/43786,280,532017112300002,محافظة المنيا المنطقة الصناعية بالمطاهرة شرق ا...,شركة اشرف ابراهيم محمد ابراهيم وشريكيه شركة اش...,1,10613.0,2017/43786,جزء من القطعة رقم 9 / بلوك ج / الخلية الثانية ...,دريم للمنتجات الورقية,1.0
79302,0.317068,532018011600006,2017/44321,352,532018011600006,المنيا - المنيا الجديدة - المنطقة الصناعية الح...,مصنع محسن رضا محمود مرسي,1,11308.0,2017/44321,مصنع مءجر بملك لبني يوسف عبد الوارث /قطعة رقم ...,مصنع اخشاب ( محسن رضا محمود مرسي ),1.0
83835,0.300466,532018021300002,2012/37415,373,532018021300002,بني سويف - المنطقة الصناعية بياض العرب - شرق ا...,بباوى ونيس عبد الشهيد ونيسكو لاند للنباتات الطبية,1,915.0,2012/37415,"قطعة رقم 85 , 86 / المنطقة الصناعية بياض العرب...",ونيسكو لاند للنباتات الطبية,1.0
106366,0.445707,532018041100001,2012/37514,474,532018041100001,محاففظة المنيا - المنطقة الصناعية بالمطاهرة - ...,مصنع كواليتى لصناعة الاسفنج ( احمد محمد عاطف...,1,3529.0,2012/37514,قطع ارقام من 31 الي 36/ المنطقة الصناعية بالمط...,مصنع كواليتي لصناعة الاسفنج,1.0
110017,0.465617,532018041600001,2013/38305,490,532018041600001,اسيوط - المنطقة الصناعية بالصفا - بني غالب - ا...,صلاح عبد الرحيم سيد وشركاه ( نيو اطلس للبترو...,1,4587.0,2013/38305,قطع ارقام 282،280،278/مدينة الصفا الصناعية /بن...,نيو اطلس للبتروكيماويات وخلط وتعبءة الزيوت ا...,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7476971,0.382692,152020070600001,2012/38164,32945,152020070600001,قطعة رقم 57 بالمنطقة الصناعية امتداد السادسة -...,مصطفى على مصطفى على غنيم - مصنع السلام للجلا...,1,706.0,2012/38164,قطعة رقم 57/امتداد المنطقة الصناعية السادسة/مد...,مصنع السلام للجلاش الالي,1.0
7480791,0.494590,152020071400001,2014/39926,32962,152020071400001,قطعة رقم (166و168) المنطقة الصناعية الاولى - م...,الاخوة للصناعات المتطورة BROTHERS COMPANY FOR ...,1,878.0,2014/39926,"قطعة رقم 166, 168/ المنطقة الصناعية الاولي/ 6 ...",الاخوة للصناعات المتطورة \r\nERS COMPANY FOR...,1.0
7489772,0.499271,152020020400001,1995/24627,33006,152020020400001,قطعة 161 بالمنطقة الصناعية الثانية - مدينة 6 ا...,الشركة المصرية البريطانية للكيماويات والمواد ا...,1,2786.0,1995/24627,قطعة رقم 161/ المنطقة الصناعية الثانية/ 6 اكتو...,المصرية البريطانية للكيماويات والمواد المساعدة...,1.0
7514242,0.419279,152020091300002,1991/19070,33117,152020091300002,ك 28 طريق مصر اسكندرية الصحراوى - طريق المحولا...,شركة موبيكا للصناعات المتكاملة شركة موبيكا للص...,1,14028.0,1991/19070,ك 28طريق مصر اسكندرية الصحراوي/ منطقة المحولات...,شركة موبيكا للصناعات المتكاملة,1.0


In [46]:
right_on= ["NameCompare_Licence", "Address_Licence"]

left_on = ["NameCompare_segl", "Address_segl"]

In [49]:
matched_resultsMin = fuzzymatcher.fuzzy_left_join(Segelcleaned,
                                            Licensecleaned,
                                            left_on,
                                            right_on,
                                            left_id_col='seg_cd',
                                            right_id_col='LicenseCode')+

In [50]:
matched_resultsMin["LicenseCode"]=matched_resultsMin["LicenseCode"].astype(str)

In [51]:
matched_resultsMin.to_excel("matched_resultsMin.xls")

In [80]:
matched_resultsMin["LicenseCode"]=matched_resultsMin["LicenseCode"].str.split("." ,expand=True).iloc[:,0]

In [82]:
matched_resultsMin.to_excel("matched_resultsMin.xls")

In [85]:
matched_resultsMin

,best_match_score,__id_left,__id_right,Unnamed: 0_left,seg_cd,Address_segl,NameCompare_segl,Key_left,Unnamed: 0_right,LicenseCode,Address_Licence,NameCompare_Licence,Key_right
0,-2.552485,2016/41930,1.320190e+14,0,2016/41930,مصنع مءجر بملك / سعيد احمد معوض عبد الباقى - ا...,واى لصناعة المطابخ والاثاث WHY FOR KITCHEN A...,1,29227.0,132019030400001,مصنع مقام على القطعة رقم ( 233) - منطقة الورش ...,محمد محمد احمد عيد ( توب انيرجى) TOP ENERGY...,1.0
217,-1.328195,2005/31258,4.320201e+14,1,2005/31258,كفر سعد/ اول طريق بنها المنصورة/ بنها/ محافظة ...,شركة لوتس لمواد التعبءة والتغليف (لوتس باك),1,32409.0,432020062300007,قطعة رقم 65 - منطقة مبارك الصناعية - قويسنا - ...,الوطنية لمواد التعبءة والتغليف ( وطن باك ),1.0
433,-1.656364,2005/31063,1.620191e+14,2,2005/31063,26 شارع سعد زغلول من شارع 135 طريق بهتيم / شبر...,كلر ميكس لصناعة وطباعة البطاطين والمنسوجات,1,1973.0,162019120100006,19 شارع سعد زغلول - شبرا البلد ملك / ورثة فوزى...,محمد مصطفى مصطفى سالم محمد مصطفى مصطفى سالم,1.0
657,-1.428991,2009/34162,1.320181e+14,3,2009/34162,مصنع مءجر مقام على القطعة رقم1/ 18/بلوك20016/ا...,مصنع بيكنج فود للصناعات الغذاءية,1,28913.0,132018061100005,مصنع مءجر على القطعة رقم 16 ب / بلوك 20014 / ا...,"عماد الدين محمد صايغ ( قطاع خاص - فردى ) "" م...",1.0
916,-0.949820,2007/32080,1.220180e+14,4,2007/32080,القطعة رقم 8/1/ الوحدة 3/ مجمع الشركة السويسري...,الشركة المصرية السويسرية للصناعة والتجارة,1,14166.0,122018010200001,القطعة رقم 1/1 ، 11/1 ، المنطقة الصناعية الراب...,الشركة المصرية الالمانية للسيارات - اجـــا الش...,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3121240,-2.225723,2018/45093,4.220181e+14,14076,2018/45093,مخبز مءجر بملك العربى عنتر حسين / ب 10 مسجد ا...,مءسسة محمد العربى,1,19453.0,422018112500016,مءجر بملك / نبيل عبد العظيم حسين - 88 شارع صدق...,محمد نبيل عبد العظيم حسين عبد العظيم,1.0
3121486,-0.774271,2020/47183,1.320190e+14,14077,2020/47183,مصنع مءجر / قطعة رقم 191 / منطقة الصناعات المت...,ديلفى لانتاج الاجهزة الكهرباءية,1,29222.0,132019022700002,قطعة رقم (252) / منطقة المتنوعة بالمنطقة الصنا...,""" اسكاروس للصناعة "" ناجى انور موسي واصف "" قط...",1.0
3121699,-0.430631,2019/45837,1.520201e+14,14078,2019/45837,قطعة رقم 212 / بقطاع الصناعات الغذاءية / بالمن...,اورجنك لايف للصناعات الغذاءية اورجنك لايف للصن...,1,33059.0,152020072700001,قطعة رقم ( جزء من 253 و جزء من 254 ) بقطاع الص...,كوول للمنتجات الغذاءية كوول للمنتجات الغذاءية,1.0
3121946,-1.335472,2007/32378,3.120190e+14,14079,2007/32378,عنبر مءجر بالدور الثاني علوي من المبني المقام ...,المصرية للالياف المعدنية المصرية للالياف المعدنية,1,17200.0,312019031900008,القطعة رقم 61 ماعدا الدور الثاني مءجر- المنطقة...,الشركة المصرية للخيوط - خالد محمد بيومي محمد...,1.0


In [87]:
matched_resultsMin[(matched_resultsMin["best_match_score"]>0)]

,best_match_score,__id_left,__id_right,Unnamed: 0_left,seg_cd,Address_segl,NameCompare_segl,Key_left,Unnamed: 0_right,LicenseCode,Address_Licence,NameCompare_Licence,Key_right
2841,0.361027,2008/33022,7.220181e+14,12,2008/33022,المنطقة الصناعية امام جمعية الغزل/ المحلة الكب...,مصنع زهرة المحلة للنسيج والصباغة,1,25885.0,722018102500001,المنطقة الصناعية امام جمعية الغزل بملك / عبد ا...,زهرة المحلة للنسيج والصباغة ( شركة مساهمة ),1.0
2966,0.485472,2013/8947,5.320180e+14,13,2013/8947,قطعة رقم 1/ بلوك ب/ المنطقة الصناعية / مدينة ا...,مصنع الالات الزراعية,1,328.0,532018010400006,المنيا - المنيا الجديدة - المنطقة الصناعية - ب...,مصنع الالات الزراعية لصاحبها صبحي يوسف عطية,1.0
5920,0.410646,2011/36794,1.320181e+14,25,2011/36794,قطعة رقم 3/ بلوك 20021/ مدينة العبور/ محافظة ا...,رالف لتصنيع الملابس الجاهزة والاستيراد والتصدير,1,28953.0,132018080600001,قطعه رقم 3 بلوك 20021 - مدينه العبور - محافظة ...,رالف لتصنيع الملابس الجاهزة والاستيراد والتص...,1.0
6788,1.588342,2013/38552,1.320181e+14,29,2013/38552,"القطع ارقام 502, 503, 507, 508/ حوض خارج الزما...",تكنولوجيا البيءة المتكاملة والخدمات البترولي...,1,28884.0,132018051400002,القطع ارقام (508،507،503،502) تقسيم منطقة الشر...,تكنولوجيا البيءة المتكامله والخدمات البترولي...,1.0
7259,0.308907,2013/38555,7.120181e+14,32,2013/38555,القطع ارقام من 1 الي 8 والقطعة رقم 13 والشارع ...,الشركة الدولية للخدمات والصناعة,1,24961.0,712018062800001,القطع ارقام من (1الى 8 ) واالقطعة 13 المنطقة ا...,"الشركة الدولية للخدمات و الصناعة "" ش . م .م """,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3113992,0.224035,2008/33207,7.220191e+14,14042,2008/33207,المندورة/مركز دسوق/محافظة كفر الشيخ,جرين فيلدز,1,26301.0,722019051900001,المندورة - مركز دسوق - كفر الشيخ,شركة جرين فيلدز ( عبد الواحد عبد الواحد سعد ...,1.0
3114613,1.195275,2019/46040,4.220180e+14,14045,2019/46040,مصنع مءجر بملك / ورثة هشام سعيد عبد العزيزيحي ...,الاسد لصناعة الطوب الاسد لصناعة الطوب,1,18448.0,422018021100003,مصنع مءجر بملك ( ورثة هشام سعيد عبد العزيز يح...,عوض مصطفى محمد محمد ناصر ( الاسد لصناعة الطوب ),1.0
3119255,0.367375,1997/25796,1.220180e+14,14066,1997/25796,"قطع 1/1 , 11/1/ المنطقة الصناعية الرابعة / 6 ا...","الشركة المصرية الالمانية للسيارات "" اجا """,1,14166.0,122018010200001,القطعة رقم 1/1 ، 11/1 ، المنطقة الصناعية الراب...,الشركة المصرية الالمانية للسيارات - اجـــا الش...,1.0
3119692,1.323724,1996/25374,1.520200e+14,14069,1996/25374,مصنع مءجر مقام علي الكيلو 35 طريق الفيوم القاه...,الفراعنة ستايل لصناعه الادوات الصحية,1,32916.0,152020031800004,مصنع مءجر مقام على الكيلو 35 طريق الفيوم القاه...,الفراعنة ستايل لصناعة الادوات الصحية الفراعنة ...,1.0


In [88]:
matched_resultsMin[(matched_resultsMin["best_match_score"]>-.3)& (matched_resultsMin["best_match_score"]<0)]

,best_match_score,__id_left,__id_right,Unnamed: 0_left,seg_cd,Address_segl,NameCompare_segl,Key_left,Unnamed: 0_right,LicenseCode,Address_Licence,NameCompare_Licence,Key_right
4488,-0.280431,2011/37232,3.320181e+14,19,2011/37232,"القطع ارقام4,3,2 / المنطقة الصناعية الخامسة / ...",عالم الاثاث العصرى,1,29751.0,332018052800001,قطع ارقم ( 2 ، 3 ، 4 ) المنطقة الصناعية الخامس...,عالم الاثاث العصرى ( عيد السعداوى وشركاه ) -...,1.0
15935,-0.029478,1982/2163,3.120180e+14,73,1982/2163,قطعة رقم Vig / المنطقة الصناعية B.I / مدينة ...,مءسسة ماريا للتجارة والصناعة,1,16065.0,312018022100005,القطعة رقم (VIG) - المنطقة الصناعية (B.1) - مد...,شركة ماريا للتجارة والصناعة (معتز هانى عمر ا...,1.0
19499,-0.243864,2008/33414,2.220180e+14,88,2008/33414,قطعة رقم12/بلوك 134/المنطقة الصناعية الرابعة/م...,المصرية الايطالية للصناعة,1,3566.0,222018041800022,قطعة 4 - بلوك 3 - المنطقة الصناعية الرابعة - م...,المصرية الدولية - رمضان محمد اسماعيل ابراهيم,1.0
20210,-0.254784,2013/38577,1.220190e+14,91,2013/38577,قطعة رقم 191/المرحلة الاولي/المنطقة الصناعية/ك...,السارة بلاست (رفعت فهمي يسي وشريكه),1,15550.0,122019040100001,"القطعه رقم (191,ضمن 192 كتوسع ) - المرحله الاو...","شركه السارة بلاست "" رفعت فهمى يس وشريكه "" ذ ...",1.0
21131,-0.229870,2013/38579,1.520201e+14,96,2013/38579,قطعة 30/ المنطقة الصناعية السادسة / 6 اكتوبر /...,مكه للصناعـــــــــة,1,33092.0,152020082400003,قطعة رقم 30 بالمنطقة الصناعية السادسة - مدينة ...,فواز موتورز - مستاجر,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3113032,-0.232924,2014/40301,4.320180e+14,14038,2014/40301,قطعة رقم 2/ منطقة مبارك الصناعية/ قويسنا/ محاف...,العربي للصناعات الهندسية,1,30955.0,432018041600001,قطعة رقم 227 - منطقة مبارك الصناعية - قويسنا -...,المصرية اللبنانية للصناعات الغذاءية ش.م.م,1.0
3117032,-0.289581,1995/24641,1.720201e+14,14056,1995/24641,18 شارع عبد العزيز جاويش/قسم عابدين/محافظة الق...,سكوب للبصريات سكوب للبصريات,1,25092.0,172020061100002,18 ش عبد العزيز جاويش - عابدين - القاهره,محمد احمد عبد العظيم محمد احمد عبد العظيم,1.0
3117501,-0.177563,2006/31852,1.620191e+14,14058,2006/31852,مصنع مءجر ملك عصام سيد محمد / ابو سنة / مجمع ...,فايمس بلاست فايمس بلاست,1,2071.0,162019121200011,مصنع مءجر بملك / عصام سيد محمد هلال - شارع اب...,محمد مختار محمد محمد عبد العال وشريكه محمد مخت...,1.0
3118599,-0.113525,1998/26447,1.320200e+14,14063,1998/26447,عين حلوان/حلوان/محافظة القاهرة,شركة حلوان للالات والمعدات,1,29544.0,132020020300001,عين حلوان / حلوان / محافظة القاهرة,شركه حلوان للصناعات الهندسيه شركه حلوان للصناع...,1.0


In [ ]:
#df[["Job", "Location"]] = df["Job"].str.split(r"\bat\b", expand=True)